In [1]:
from scipy.stats import beta
import operator
import numpy as np
from scipy.special import betaln

In [2]:
dataset = 1
if dataset==0:
    # MovieLens-1M dataset
    ratings_file = '../data/ml-1m/ratings.dat'
    delimiter = '::'
elif dataset==1:
    # MovieLens-100k dataset
    ratings_file = '../data/ml-100k/u.data'
    delimiter = '\t'

In [32]:
def load_1_m():
    ratings = open(ratings_file, 'r').read().split('\n')
    user_item_map = {}
    for r in ratings:
        attrs = r.split(delimiter)
        if len(attrs) < 4:
            continue
        user = 'u' + attrs[0]
        item = 'i' + attrs[1]
        rating = int(attrs[2])
        if user in user_item_map:
            user_item_map[user][item] = rating
        else:
            user_item_map[user] = {}
            user_item_map[user][item] = rating
    for user in user_item_map:
        sum = 0
        for item in user_item_map[user]:
            sum += user_item_map[user][item]
        avg_rating_user = sum * 1.0 / len(user_item_map[user])
        for item in user_item_map[user]:
            if user_item_map[user][item] >= avg_rating_user:
                user_item_map[user][item] = 1
            else:
                user_item_map[user][item] = 0
    for user in user_item_map:
        if len(user_item_map[user]) < 10:
            del user_item_map[user]
    return user_item_map

In [33]:
def form_graph(user_item_map):
    graph = {}
    for user in user_item_map:
        if user not in graph:
            graph[user] = set([])
        for item in user_item_map[user]:
            if item not in graph:
                graph[item] = set([])
            graph[user].add(item)
            graph[item].add(user)
    return graph

In [34]:
def clean_graph(graph):
    while True:
        changed = False
        delete_nodes = []
        for node in graph:
            if len(graph[node]) < 10:
                changed = True
                delete_nodes.append(node)
        for node in delete_nodes:
            del graph[node]
        for node1 in graph:
            delete_nodes = []
            for node2 in graph[node1]:
                if node2 not in graph:
                    changed = True
                    delete_nodes.append(node2)
            for node2 in delete_nodes:
                graph[node1].remove(node2)
        if not changed:
            break
    for node in graph:
        graph[node] = list(graph[node])
    return graph

In [35]:
def get_num_ratings(user_item_map):
    item_rating_map = {}
    for user in user_item_map:
        for item in user_item_map[user]:
            if item not in item_rating_map:
                item_rating_map[item] = [1, 1]
            if user_item_map[user][item] == 0:
                item_rating_map[item][1] += 1
            else:
                item_rating_map[item][0] += 1
    return item_rating_map

In [36]:
user_item_map = load_1_m()

In [37]:
item_rating_map = get_num_ratings(user_item_map)

In [38]:
graph = form_graph(user_item_map)

In [39]:
graph = clean_graph(graph)

In [40]:
len(graph.keys())

2095

## Split MovieLens-100k data so that for each user, 80% is training, 20% is for test


In [41]:
first_20percent_of_u1 = [graph['u1'][i] for i in range(int(0.2*len(graph['u1'])))] 
print first_20percent_of_u1

['i148', 'i149', 'i142', 'i143', 'i140', 'i141', 'i146', 'i147', 'i144', 'i145', 'i177', 'i229', 'i118', 'i228', 'i270', 'i271', 'i272', 'i89', 'i205', 'i82', 'i83', 'i80', 'i81', 'i86', 'i87', 'i84', 'i85', 'i153', 'i97', 'i259', 'i193', 'i77', 'i76', 'i73', 'i72', 'i71', 'i70', 'i189', 'i79', 'i78', 'i204', 'i252', 'i158', 'i263', 'i262', 'i261', 'i260', 'i266', 'i265', 'i264', 'i269', 'i268']


In [42]:
len(graph['u1'])

263

In [43]:
graph['u1'][3]

'i143'

In [44]:
# Input: a graph called g
# Output: two test sets, one containing 20% of g's items per user, 
#         the other containing 80% 

from collections import defaultdict

def create_sets(g): 
    test_set = defaultdict(list) 
    training_set = defaultdict(list)
    
    for key in g:
        if key[0]=='u': 
            # For length of items belonging to that key, split 20% and 80%
            first_20 = int(0.2*len(g[key]))
            
            for i in range(first_20): 
                test_set[key].append(g[key][i])

            for j in range(first_20, len(g[key])):
                training_set[key].append(g[key][j])
            
    for user in test_set.keys():
        for item in test_set[user]:
            test_set[item].append(user)
            
    for user in training_set.keys(): 
        for item in training_set[user]: 
            training_set[item].append(user)
            
    return test_set, training_set

In [55]:
test_set, training_set = create_sets(graph)

In [46]:
print first_20percent_of_u1 == test_set['u1']

True


In [47]:
print "movie_100k_test_set has ", len(test_set.keys()), "keys"
print "movie_100k_training_set has ", len(training_set.keys()), " keys"

movie_100k_test_set has  1833 keys
movie_100k_training_set has  2094  keys


In [57]:
def find_user_similarity(graph):
    users = []
    for key in graph:
        if 'u' in key:
            users.append(key)
    similarity = np.zeros((len(users) + 1, len(users) + 1))
    count = np.zeros((len(users) + 1, len(users) + 1))
    np.fill_diagonal(similarity, 1)
    np.fill_diagonal(count, 1)
    for u1 in users:
        for u2 in users:
            if u1 == u2:
                continue
            for item in graph[u1]:
                if item in graph[u2]:
                    count[int(u1[1:]), int(u2[1:])] += 1
                    if user_item_map[u1][item] == user_item_map[u2][item]:
                        similarity[int(u1[1:]), int(u2[1:])] += 1
                    else:
                        similarity[int(u1[1:]), int(u2[1:])] -= 1
            count[int(u2[1:]), int(u1[1:])] = count[int(u1[1:]), int(u2[1:])]
            similarity[int(u2[1:]), int(u1[1:])] = similarity[int(u1[1:]), int(u2[1:])]
    return np.divide(similarity, count)

In [58]:
user_similarity = find_user_similarity(training_set)

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in divide


In [59]:
user_similarity[np.isnan(user_similarity)] = 0
min_value = user_similarity.min()
max_value = user_similarity.max()
user_similarity = (user_similarity - min_value) / (max_value - min_value)

In [60]:
print len(np.argwhere(user_similarity[:, :] > 0.5)) * 1.0 / (944**2)
print min_value
print max_value

0.543638681413
-1.0
1.0


In [61]:
PIS_map = {}
PPS_map = {}
PORS_map = {}

In [62]:
def PIS(item_pair):
    item1 = int(item_pair[0][1:])
    item2 = int(item_pair[1][1:])
    total = 0
    for i in range(0,item_rating_map[item2][0]):
        total += np.exp(betaln(item_rating_map[item1][0]+i,item_rating_map[item1][1]+item_rating_map[item2][1]) -\
                        np.log(item_rating_map[item2][1]+i) - \
                        betaln(1+i, item_rating_map[item2][1]) -\
                        betaln(item_rating_map[item1][0],item_rating_map[item1][1])
                       )
    return total

In [63]:
def PPS(item_pair):
    item1 = int(item_pair[0][1:])
    item2 = int(item_pair[1][1:])
    p1 = (item_rating_map[item1][0]) * 1.0 / (item_rating_map[item1][0] + item_rating_map[item1][1])
    p2 = (item_rating_map[item2][0]) * 1.0 / (item_rating_map[item2][0] + item_rating_map[item2][1])
    return p1 * p2

In [64]:
def PORS(item_pair):
    item1 = int(item_pair[0][1:])
    item2 = int(item_pair[1][1:])
    o1 = (item_rating_map[item1][0]) * 1.0 / (item_rating_map[item1][1])
    o2 = (item_rating_map[item2][0]) * 1.0 / (item_rating_map[item2][1])
    return o2 / o1

In [67]:
def USS(u1, i1, u2, i2):
    reliability = item_rating_map[int(i2[1:])][0] * 1.0 / (item_rating_map[int(i2[1:])][0] + item_rating_map[int(i2[1:])][1])
    indicator = 1
    if user_item_map[int(u2[1:])][int(i2[1:])] == 0:
        indicator = 0
    similarity = user_similarity[int(u1[1:]), int(u2[1:])]
    return reliability * similarity * indicator + reliability * (1.0 - similarity) * (1 - indicator)

In [66]:
def rank(graph, target_user):
    score_PIS = {}
    score_PPS = {}
    score_PORS = {}
    score_USS = {}
    PIS_values = {}
    PPS_values = {}
    PORS_values = {}
    USS_values = {}

In [72]:
def rank(graph, target_user):
    score_map_PPS = {}
    score_map_PORS = {}
    score_map_PIS = {}
    score_map_user_similarity = {}
    for primary_item in graph[target_user]:
        score_map_PPS[primary_item] = 0.0
        score_map_PORS[primary_item] = 0.0
        score_map_PIS[primary_item] = 0.0
        for secondary_user in graph[primary_item]:
            if secondary_user == target_user:
                continue
            for secondary_item in graph[secondary_user]:
                if secondary_item in graph[target_user]:
                    continue
                if (primary_item, secondary_item) in PIS_map:
                    score_map_PIS[primary_item] += PIS_map[(primary_item, secondary_item)]
                else:
                    PIS_map[(primary_item, secondary_item)] = PIS((primary_item, secondary_item))
                    score_map_PIS[primary_item] += PIS_map[(primary_item, secondary_item)]
                if (primary_item, secondary_item) in PPS_map:
                    score_map_PPS[primary_item] += PPS_map[(primary_item, secondary_item)]
                else:
                    PPS_map[(primary_item, secondary_item)] = PPS((primary_item, secondary_item))
                    score_map_PPS[primary_item] += PPS_map[(primary_item, secondary_item)]
                if (primary_item, secondary_item) in PORS_map:
                    score_map_PORS[primary_item] += PORS_map[(primary_item, secondary_item)]
                else:
                    PORS_map[(primary_item, secondary_item)] = PORS((primary_item, secondary_item))
                    score_map_PORS[primary_item] += PORS_map[(primary_item, secondary_item)]
                if (secondary_user, secondary_item) in score_map_user_similarity:
                    s
                x = user_similarity_score(target_user, primary_item, secondary_user, secondary_item)
    return score_map_PIS, score_map_PPS, score_map_PORS

In [73]:
#ranking = rank(graph, 'u1')
rank(training_set, 'u1')

In [ ]:
ranking_PIS = ranking[0]
ranking_PPS = ranking[1]
ranking_PORS = ranking[2]

In [ ]:
sorted_1 = sorted(ranking_PIS.items(), key=operator.itemgetter(1))
sorted_2 = sorted(ranking_PPS.items(), key=operator.itemgetter(1))
sorted_3 = sorted(ranking_PORS.items(), key=operator.itemgetter(1))

print sorted_1[:5]
print sorted_2[:5]
print sorted_3[:5]

In [ ]:
print ranking_PIS

## Generate 3 lists of ranking scores per item for PIS, PPS, PORS

In [ ]:
# ******Caution: the code in this box will need to run overnight due to the size of the test_set****** 

# Input:  One of the score dictionaries generated by calling rank(graph, key) 
# Output: A ranking list for each user, where the most highly recommended items come first (no need to store
#         the score that was associated with that item)
def generate_ranking_list(score): 
    ranking_list = {} 

    for user in score: 
        # Sort the score list for that user 
        sorted_score = sorted(score[user].items(), key=operator.itemgetter(1), reverse=True)
        ranking_list[user]= [item[0] for item in sorted_score]

    return ranking_list 

# For each of the 3 methods, generate scores as a dictionary where 
# Key = User, Value = List of (item,score) pairs 
score_PIS = {} 
score_PPS = {} 
score_PORS = {}
score_user_similarity = {}

for user in training_set: 
    if user[0] == 'u':
        score_PIS[user], score_PPS[user], score_PORS[user], score_user_similarity = rank(graph, key)
        
# Generate 3 lists of ranking scores per item for PIS, PPS, PORS
ranking_PIS = generate_ranking_list(score_PIS)
ranking_PPS = generate_ranking_list(score_PPS)
ranking_PORS = generate_ranking_list(score_PORS)
ranking_user_similarity = generate_ranking_list(score_user_similarity)

In [ ]:
def get_lp_ln(predictions):
    lp = defaultdict{list}
    ln = defaultdict{list}
    for user in predictions:
        for item in testing_set[user]:
            if user_item_map[user][item] == 1:
                lp[user].append(item)
            else:
                ln[user].append(item)
    return lp,ln

def p_at_k(k, predictions, lp):
    patk = 0.0
    for user in predictions:
        correct = 0
        for i in range(k):
            if predictions[user][i] in lp[user]:
                correct+=1
        patk += 1.0*correct/k
    return patk/len(predictions.keys())

def MAP(predictions, lp):
    MAP = 0.0
    for user in predictions:
        umap = 0.0
        for i in range(len(predictions[user])):
            if predictions[user][i] in lp:
                correct += 1
                umap += 1.0*correct/(i*len(lp[user]))
    MAP += umap
    return MAP/len(predictions.keys())

def MRR(predictions, lp):
    MRR = 0.0
    for user in predictions:
        for i in range(len(predictions[user])):
            if predictions[user][i] in lp:
                MRR += 1.0/i
                break
    return MRR/len(predictions.keys())

def rel(item, u_lp, u_ln):
    if item in u_lp:
        return 2
    elif item in u_ln:
        return 1
    else:
        return 0

def dcg_at_n(n, u_pred, u_lp, u_ln):
    u_dcg = 0.0
    for i in range(n):
        if i==0:
            u_dcg += rel(u_pred[i], u_lp, u_ln)
        else:
            u_dcg += rel(u_pred[i], u_lp, u_ln)/np.log2(i)
    return u_dcg
    
def idcg_at_n(n, u_pred, u_lp, u_ln):
    rel_scores = []
    for item in u_pred:
        rel_scores.append(rel(item, u_lp, u_ln))
    rel_scores = sorted(rel_scores, reverse=True)
    u_idcg = rel_scores[0]
    for i in range(1,len(rel_scores)):
        u_idcg += rel_scores[i]/np.log2(i)
    return u_idcg
        
def ndcg_at_n(n, predictions, lp, ln):
    ndcg = 0.0
    for user in predictions:
        u_dcg = dcg_at_n(n, predictions[user], lp[user], ln[user])
        u_idcg = idcg_at_n(n, predictions[user], lp[user], ln[user])
        ndcg += u_dcg/u_idcg
    return ndcg/len(predictions.keys())

k = 10
n = 10
lp, ln = get_lp_ln(predictions)
print p_at_k(k, predictions, lp)
print MAP(predictions, lp)
print MRR(predictions, lp)
print ndcg_at_n(n, predictions, lp, ln)